<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/mbp_mins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 술 중 혈압에 따른 MINS 발생 위험
본 예제에서는 오픈 데이터셋인 vitaldb 를 이용하여 술 중 혈압에 따른 술 후 심근 손상 (myocardial injury after non-cardiac surgery, MINS) 발생 위험을 구해보자.

## 필요 라이브러리 및 데이터 다운로드

In [1]:
import pandas as pd
import numpy as np

df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # 임상 정보
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # 트랙 목록
df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # lab 데이터

## 본 예제에서 사용할 case 선택
vitaldb 환자 중 troponin I 결과가 있는 환자 100명만 사용해보자.

In [2]:
caseids = list(
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']['caseid']) & 
    set(df_labs[df_labs['name'] == 'Troponin I']['caseid'])
)
caseids = caseids[:100]
print('Total {} cases found'.format(len(caseids)))

Total 100 cases found


## 각 케이스별 데이터 받아오기 및 계산
vitaldb 로부터 각 case별 술 후 troponin I 농도를 구해보고 이로부터 MINS 발생 여부를 구해보자. 본 예제에서 MINS 의 정의는 troponin I > 0.028 ng/mL로 정의한다.
또한 수술 중 ART_MBP 데이터를 받아와 40-80 mmHg 의 각 threshold 들에 대해 전체 수술 중 해당 threshold  이하에 머무른 측정치의 비율을 구해보자.

In [ ]:
# 혈압 기준치
mbp_thresholds = np.arange(40, 80)

# 최종 결과 저장
df = pd.DataFrame()
for caseid in caseids:
    print('loading {}...'.format(caseid), flush=True, end='')

    # 마취 종료 시각을 가져옴
    aneend = df_cases[(df_cases['caseid'] == caseid)]['aneend'].values[0]

    # 술 후 48 시간 이내 최대 creatinine 농도
    postop_tpi = df_labs[(df_labs['caseid'] == caseid) & (df_labs['dt'] > aneend) &
        (df_labs['dt'] < aneend + 48 * 3600) & (df_labs['name'] == 'Troponin I')]['result'].max(skipna=True)
    if not postop_tpi or np.isnan(postop_tpi):
        print('no postop troponin I')
        continue

    # mins = postop_tpi > 0.028
    mins = postop_tpi > 0.028

    # 술 중 혈압
    tid_mbp = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Solar8000/ART_MBP')]['tid'].values[0]
    mbps = pd.read_csv('https://api.vitaldb.net/' + tid_mbp).values[:,1]
    mbps = mbps[~np.isnan(mbps)]
    mbps = mbps[(mbps > 20) & (mbps < 150)]
    if len(mbps) < 10:
        print('no mbp')
        continue

    # 수술 중 혈압을 1단위로 증가시키면서 해당 시간 동안 머무른 비율을 구함
    row = {'mins':mins}
    for mbp_threshold in mbp_thresholds:
        row['under{}'.format(mbp_threshold)] = np.nanmean(mbps < mbp_threshold) * 100

    # 결과 행에 추가
    df = df.append(row, ignore_index=True)

    print('{}, {}'.format(postop_tpi, 'MINS' if mins else ''))

print('{} MINS {:.1f}%'.format(df['mins'].sum(), df['mins'].mean() * 100))

loading 1...0.01, 
loading 4...0.01, 
loading 6152...0.1, MINS
loading 10...0.69, MINS
loading 2058...0.01, 
loading 12...0.07, MINS
loading 2060...0.01, 
loading 4109...no postop troponin I
loading 2063...0.02, 
loading 2064...0.01, 
loading 17...0.01, 
loading 6156...0.01, 
loading 19...no postop troponin I
loading 20...0.02, 
loading 4115...0.03, MINS
loading 6159...no postop troponin I
loading 6166...0.01, 
loading 25...0.02, 
loading 6174...0.01, 
loading 4128...0.01, 
loading 2082...0.01, 
loading 6180...0.01, 
loading 2085...no postop troponin I
loading 4135...0.02, 
loading 6185...no postop troponin I
loading 6186...0.01, 
loading 4143...0.01, 
loading 2097...no postop troponin I
loading 4146...0.01, 
loading 2100...0.01, 
loading 4148...

## 각 혈압 기준치별로 MINS 예측에 대한 odd ratio를 구함
case 별 MINS 발생 여부 및 각 threshold 에 머무른 비율을 이용하여 각 혈압 기준치가 MINS 발생 risk 를 얼마나 증가시키는지 odd ratio 를 구해보자.

In [ ]:
# univariate logistic regression을 돌려 odd ratio 를 구함
import statsmodels.api as sm
df['intercept'] = 1
df['mins'] = df['mins'].astype(bool)
odd_ratios = []
for mbp_threshold in mbp_thresholds:
    c = 'under{}'.format(mbp_threshold)
    model = sm.Logit(df['mins'], df[['intercept', c]])
    res = model.fit()
    b = res.params[c]
    pval = res.pvalues[c]
    odd_ratios.append(np.exp(b))
    print('{}\tb={:.3f}, exp(b)={:.3f}, pval={:.3f}'.format(c, b, np.exp(b), pval))

## 결과를 그림으로
대략 MBP가 낮게 머무르는 시간이 늘어날 수록 MINS 의 odd ratio가 증가함을 알 수 있다.

In [ ]:
# 결과를 그림으로
import matplotlib.pyplot as plt
plt.plot(mbp_thresholds, odd_ratios)
plt.show()